# Unit test writing using a multi-step prompt

Complex tasks, such as writing unit tests, can benefit from multi-step prompts. In contrast to a single prompt, a multi-step prompt generates text from GPT and then feeds that output text back into subsequent prompts. This can help in cases where you want GPT to reason things out before answering, or brainstorm a plan before executing it.

In this notebook, we use a 3-step prompt to write unit tests in Python using the following steps:

1. **Explain**: Given a Python function, we ask GPT to explain what the function is doing and why.
2. **Plan**: We ask GPT to plan a set of unit tests for the function.
    - If the plan is too short, we ask GPT to elaborate with more ideas for unit tests.
3. **Execute**: Finally, we instruct GPT to write unit tests that cover the planned cases.

The code example illustrates a few embellishments on the chained, multi-step prompt:

- Conditional branching (e.g., asking for elaboration only if the first plan is too short)
- The choice of different models for different steps
- A check that re-runs the function if the output is unsatisfactory (e.g., if the output code cannot be parsed by Python's `ast` module)
- Streaming output so that you can start reading the output before it's fully generated (handy for long, multi-step outputs)

In [1]:
!pwd

/Users/patrick.schneider/Library/CloudStorage/OneDrive-RaiffeisenBankInternationalGroup/hackathon/adc-hackathon-2024-05-t1/workbooks


In [10]:
# import sys
# sys.path.append("..")
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
# import sys
# import os


In [28]:
import importlib
import subprocess
import sys

# Read the 'requirements.txt' file and extract package names
with open('../requirements.txt', 'r') as file:
    package_names = [line.strip() for line in file]

# Import the packages dynamically
for package_name in package_names:
    try:
        importlib.import_module(package_name)
    except ImportError:
        print(f"Failed to import '{package_name}'. Attempting to install...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        try:
            importlib.import_module(package_name)
            print(f"Successfully installed and imported '{package_name}'")
        except ImportError:
            print(f"Failed to install '{package_name}'. Please install it manually.")

In [30]:
# Add the path to the 'src' directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

import unittest_python as ut
importlib.reload(ut)
import ast
from openai import OpenAI
import pytest

In [29]:
# import ast
# from openai import OpenAI
# import pytest

# color_prefix_by_role = {
#     "system": "\033[0m",  # gray
#     "user": "\033[0m",  # gray
#     "assistant": "\033[92m",  # green
# }

# def print_messages(messages, color_prefix_by_role=color_prefix_by_role):
#     """
#     Print messages sent to or from GPT.

#     Parameters:
#     - messages: List of messages to print.
#     - color_prefix_by_role (dict, optional): Color prefixes for different roles (default is provided color scheme).

#     Prints messages with color-coded prefixes.
#     """
#     for message in messages:
#         role = message["role"]
#         color_prefix = color_prefix_by_role[role]
#         content = message["content"]
#         print(f"{color_prefix}\n[{role}]\n{content}")


# def print_message_delta(delta, color_prefix_by_role=color_prefix_by_role):
#     """
#     Print a chunk of messages streamed back from GPT.

#     Parameters:
#     - delta: The message delta to print.
#     - color_prefix_by_role (dict, optional): Color prefixes for different roles (default is provided color scheme).

#     Prints message chunks with color-coded prefixes.
#     """
#     if "role" in delta:
#         role = delta["role"]
#         color_prefix = color_prefix_by_role[role]
#         print(f"{color_prefix}\n[{role}]\n", end="")
#     elif "content" in delta:
#         content = delta["content"]
#         print(content, end="")
#     else:
#         pass

In [37]:
# !source /Users/patrick.schneider/anaconda3/envs/base/etc/conda/activate.d/env_vars.sh
# %env OPENAI_API_KEY=777
# !echo $OPENAI_API_KEY

In [33]:
unit_test_package: str = "pytest"
approx_min_cases_to_cover: int = 10
print_text: bool = True
explain_model: str = "gpt-3.5-turbo"
plan_model: str = "gpt-3.5-turbo"
execute_model: str = "gpt-3.5-turbo"
temperature: float = 0.4
reruns_if_fail: int = 1
function_to_test = """def pig_latin(text):
    def translate(word):
        vowels = 'aeiou'
        if word[0] in vowels:
            return word + 'way'
        else:
            consonants = ''
            for letter in word:
                if letter not in vowels:
                    consonants += letter
                else:
                    break
            return word[len(consonants):] + consonants + 'ay'

    words = text.lower().split()
    translated_words = [translate(word) for word in words]
    return ' '.join(translated_words)
"""

In [34]:
client = OpenAI()
# Step 1: Generate an explanation of the function

# create a markdown-formatted message that asks GPT to explain the function, formatted as a bullet list
explain_system_message = {
    "role": "system",
    "content": "You are a world-class Python developer with an eagle eye for unintended bugs and edge cases. You carefully explain code with great detail and accuracy. You organize your explanations in markdown-formatted, bulleted lists."
}
explain_user_message = {
    "role": "user",
    "content": f"""Please explain the following Python function. Review what each element of the function is doing precisely and what the author's intentions may have been. Organize your explanation as a markdown-formatted, bulleted list.

```python
{function_to_test}
```""",
    }

In [36]:
explain_messages = [explain_system_message, explain_user_message]
if print_text:
    ut.print_messages(explain_messages)

explanation_response = client.chat.completions.create(
    model=explain_model,
    messages=explain_messages,
    temperature=temperature,
    stream=False
)
print('explanation_response FINISHED')
explanation = ""
if stream:
    for chunk in explanation_response:
            delta = chunk.choices[0].delta
            if print_text:
                print_message_delta(delta)
            if "content" in delta:
                explanation += delta["content"]
    explain_assistant_message = {"role": "assistant", "content": explanation}
# else:
    


[system]
You are a world-class Python developer with an eagle eye for unintended bugs and edge cases. You carefully explain code with great detail and accuracy. You organize your explanations in markdown-formatted, bulleted lists.

[user]
Please explain the following Python function. Review what each element of the function is doing precisely and what the author's intentions may have been. Organize your explanation as a markdown-formatted, bulleted list.

```python
def pig_latin(text):
    def translate(word):
        vowels = 'aeiou'
        if word[0] in vowels:
            return word + 'way'
        else:
            consonants = ''
            for letter in word:
                if letter not in vowels:
                    consonants += letter
                else:
                    break
            return word[len(consonants):] + consonants + 'ay'

    words = text.lower().split()
    translated_words = [translate(word) for word in words]
    return ' '.join(translated_words)

APIConnectionError: Connection error.

In [45]:
explanation = ""
for chunk in explanation_response:
         tmp_list.append(chunk.choices[0])
#         if print_text:
#             print_message_delta(delta)
#         if "content" in delta:
#             explanation += delta["content"]
# explain_assistant_message = {"role": "assistant", "content": explanation}

AttributeError: 'tuple' object has no attribute 'choices'

In [63]:
print(explanation_response.choices[0].message.content)

Here is an explanation of the `pig_latin` function:

- The `pig_latin` function takes a string `text` as input.
- Inside the `pig_latin` function, there is a nested function called `translate`. This function takes a string `word` as input and is responsible for translating a single word into Pig Latin.
- The `vowels` variable is a string that contains all the vowels ('a', 'e', 'i', 'o', 'u').
- The `translate` function checks if the first letter of the `word` is a vowel. If it is, it appends the string 'way' to the end of the `word` and returns it. This is the rule for translating words that start with a vowel in Pig Latin.
- If the first letter of the `word` is not a vowel, the `translate` function initializes an empty string called `consonants`. It then iterates over each letter in the `word`.
- Inside the loop, the `translate` function checks if the current letter is not in the `vowels` string. If it is not, it appends the letter to the `consonants` string. This allows the function 

In [3]:
example_function = """def pig_latin(text):
    def translate(word):
        vowels = 'aeiou'
        if word[0] in vowels:
            return word + 'way'
        else:
            consonants = ''
            for letter in word:
                if letter not in vowels:
                    consonants += letter
                else:
                    break
            return word[len(consonants):] + consonants + 'ay'

    words = text.lower().split()
    translated_words = [translate(word) for word in words]
    return ' '.join(translated_words)
"""

unit_tests = ut.unit_tests_from_function(
    example_function,
    approx_min_cases_to_cover=10,
    print_text=True
)



[system]
You are a world-class Python developer with an eagle eye for unintended bugs and edge cases. You carefully explain code with great detail and accuracy. You organize your explanations in markdown-formatted, bulleted lists.

[user]
Please explain the following Python function. Review what each element of the function is doing precisely and what the author's intentions may have been. Organize your explanation as a markdown-formatted, bulleted list.

```python
def pig_latin(text):
    def translate(word):
        vowels = 'aeiou'
        if word[0] in vowels:
            return word + 'way'
        else:
            consonants = ''
            for letter in word:
                if letter not in vowels:
                    consonants += letter
                else:
                    break
            return word[len(consonants):] + consonants + 'ay'

    words = text.lower().split()
    translated_words = [translate(word) for word in words]
    return ' '.join(translated_words)

In [4]:
print(unit_tests)

import pytest

# function to test
def pig_latin(text):
    def translate(word):
        vowels = 'aeiou'
        if word[0] in vowels:
            return word + 'way'
        else:
            consonants = ''
            for letter in word:
                if letter not in vowels:
                    consonants += letter
                else:
                    break
            return word[len(consonants):] + consonants + 'ay'

    words = text.lower().split()
    translated_words = [translate(word) for word in words]
    return ' '.join(translated_words)


# unit tests
@pytest.mark.parametrize('text, expected', [
    # Words starting with a vowel
    ('apple', 'appleway'),
    ('elephant', 'elephantway'),
    
    # Words starting with a consonant
    ('banana', 'ananabay'),
    ('cherry', 'errychay'),
    
    # Words with multiple consonants at the beginning
    ('string', 'ingstray'),
    ('glove', 'oveglay'),
    
    # Words with uppercase letters
    ('Python', 'onPythay'),
  

In [5]:
import pytest

# function to test
def pig_latin(text):
    def translate(word):
        vowels = 'aeiou'
        if word[0] in vowels:
            return word + 'way'
        else:
            consonants = ''
            for letter in word:
                if letter not in vowels:
                    consonants += letter
                else:
                    break
            return word[len(consonants):] + consonants + 'ay'

    words = text.lower().split()
    translated_words = [translate(word) for word in words]
    return ' '.join(translated_words)


# unit tests
@pytest.mark.parametrize('text, expected', [
    # Words starting with a vowel
    ('apple', 'appleway'),
    ('elephant', 'elephantway'),
    
    # Words starting with a consonant
    ('banana', 'ananabay'),
    ('cherry', 'errychay'),
    
    # Words with multiple consonants at the beginning
    ('string', 'ingstray'),
    ('glove', 'oveglay'),
    
    # Words with uppercase letters
    ('Python', 'onPythay'),
    ('JuMP', 'uMPJay'),
    
    # Words with punctuation marks
    ('hello!', 'ellohay!'),
    ('world,', 'orldway,'),
    
    # Sentences or phrases with multiple words
    ('Hello world', 'ellohay orldway'),
    ('Python is fun', 'onPythay isway unfay'),
    
    # Empty input
    ('', ''),
    
    # Input with leading or trailing spaces
    ('  hello  ', 'ellohay'),
    ('   Python is cool   ', 'onPythay isway oolcay'),
    
    # Input with numbers or special characters
    ('123', '123'),
    ('@#$', '@#$'),
    
    # Input with non-English characters
    ('éclair', 'éclairway'),
    ('über', 'überway'),
    
    # Rare or unexpected edge cases
    # Empty word
    (' ', ' '),
    
    # Word with only consonants
    ('rhythm', 'ythmrhay'),
    ('brr', 'rrbay'),
    
    # Word with only vowels
    ('ai', 'aiway'),
    ('oo', 'ooway'),
    
    # Word with alternating consonants and vowels
    ('ababa', 'ababaay'),
    ('oxoxox', 'oxoxoxway'),
    
    # Word with repeated consonants
    ('letter', 'etterlay'),
    ('grass', 'assgray'),
    
    # Word with repeated vowels
    ('moon', 'oonmay'),
    ('see', 'eesay'),
    
    # Word with a single letter
    ('a', 'away'),
    ('i', 'iway'),
    
    # Word with non-alphabetic characters
    ('123abc', '123abc'),
    ('!@#$', '!@#$'),
    
    # Long words
    ('supercalifragilisticexpialidocious', 'upercalifragilisticexpialidocioussay'),
    ('antidisestablishmentarianism', 'antidisestablishmentarianismway'),
    
    # Very long input
    ('This is a very long sentence with many words.', 'isThay isway away eryvay onglay entencesay ithway anymay ordsway.'),
    ('Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed non risus.', 'oremLay ipsumway olorday itsay ametway, onsecteturcay adipiscingway elitway. edSay onway isusray.')
])
def test_pig_latin(text, expected):
    assert pig_latin(text) == expected

Make sure to check any code before using it, as GPT makes plenty of mistakes (especially on character-based tasks like this one). For best results, use the most powerful model (GPT-4, as of May 2023).